In [1]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.svm import SVR
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv("C:\\Users\\USER\\Desktop\\car data (1).csv")

In [3]:
df.head(5)

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


In [5]:
df.isnull().sum()

Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Kms_Driven       0
Fuel_Type        0
Seller_Type      0
Transmission     0
Owner            0
dtype: int64

In [6]:
duplicates = df.duplicated()

In [7]:
print(df[duplicates])

    Car_Name  Year  Selling_Price  Present_Price  Kms_Driven Fuel_Type  \
17    ertiga  2016           7.75          10.79       43000    Diesel   
93  fortuner  2015          23.00          30.61       40000    Diesel   

   Seller_Type Transmission  Owner  
17      Dealer       Manual      0  
93      Dealer    Automatic      0  


In [8]:
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
296    False
297    False
298    False
299    False
300    False
Length: 301, dtype: bool

In [9]:
df = df.drop_duplicates(keep='first')

In [10]:
df.duplicated().sum()

0

In [11]:
# ouliers detection

def find_outliers_IQR(df):

   q1=df.quantile(0.25)

   q3=df.quantile(0.75)

   IQR=q3-q1

   outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

   return outliers

In [12]:
outliers = find_outliers_IQR(df['Selling_Price'])
print('number of outliers: '+ str(len(outliers)))
print('max outlier value: '+ str(outliers.max()))
print('min outlier value: '+ str(outliers.min()))

number of outliers: 16
max outlier value: 35.0
min outlier value: 14.25


In [13]:
# Removing Outliers
Q1 = df['Selling_Price'].quantile(0.25)
Q3 = df['Selling_Price'].quantile(0.75)
IQR = Q3 - Q1
whisker_width = 1.5
lower_whisker = Q1 -(whisker_width*IQR)
upper_whisker = Q3 +(whisker_width*IQR)
df['Selling_Price']=np.where(df['Selling_Price']>upper_whisker,upper_whisker,np.where(df['Selling_Price']<lower_whisker,lower_whisker,df['Selling_Price']))

In [14]:
Q1 = df['Selling_Price'].quantile(0.10)
Q3 = df['Selling_Price'].quantile(0.90)
IQR = Q3 - Q1
whisker_width = 1.5
lower_whisker = Q1 - (whisker_width*IQR)
upper_whisker = Q3 + (whisker_width*IQR)
index=df['Selling_Price'][(df['Selling_Price']>upper_whisker)|(df['Selling_Price']<lower_whisker)].index
df.drop(index,inplace=True)

In [15]:
outliers = find_outliers_IQR(df['Selling_Price'])
print('number of outliers: '+ str(len(outliers)))
print('max outlier value: '+ str(outliers.max()))
print('min outlier value: '+ str(outliers.min()))

number of outliers: 0
max outlier value: nan
min outlier value: nan


In [16]:
outliers = find_outliers_IQR(df['Kms_Driven'])
print('number of outliers: '+ str(len(outliers)))
print('max outlier value: '+ str(outliers.max()))
print('min outlier value: '+ str(outliers.min()))

number of outliers: 8
max outlier value: 500000
min outlier value: 104707


In [17]:
Q1 = df['Kms_Driven'].quantile(0.25)
Q3 = df['Kms_Driven'].quantile(0.75)
IQR = Q3 - Q1
whisker_width = 1.5
lower_whisker = Q1 -(whisker_width*IQR)
upper_whisker = Q3 +(whisker_width*IQR)
df['Kms_Driven']=np.where(df['Kms_Driven']>upper_whisker,upper_whisker,np.where(df['Kms_Driven']<lower_whisker,lower_whisker,df['Kms_Driven']))

In [18]:
Q1 = df['Kms_Driven'].quantile(0.10)
Q3 = df['Kms_Driven'].quantile(0.90)
IQR = Q3 - Q1
whisker_width = 1.5
lower_whisker = Q1 - (whisker_width*IQR)
upper_whisker = Q3 + (whisker_width*IQR)
index=df['Kms_Driven'][(df['Kms_Driven']>upper_whisker)|(df['Kms_Driven']<lower_whisker)].index
df.drop(index,inplace=True)

In [19]:
outliers = find_outliers_IQR(df['Kms_Driven'])
print('number of outliers: '+ str(len(outliers)))
print('max outlier value: '+ str(outliers.max()))
print('min outlier value: '+ str(outliers.min()))

number of outliers: 0
max outlier value: nan
min outlier value: nan


In [20]:
outliers = find_outliers_IQR(df['Present_Price'])
print('number of outliers: '+ str(len(outliers)))
print('max outlier value: '+ str(outliers.max()))
print('min outlier value: '+ str(outliers.min()))

number of outliers: 14
max outlier value: 92.6
min outlier value: 22.83


In [21]:
Q1 = df['Present_Price'].quantile(0.25)
Q3 = df['Present_Price'].quantile(0.75)
IQR = Q3 - Q1
whisker_width = 1.5
lower_whisker = Q1 -(whisker_width*IQR)
upper_whisker = Q3 +(whisker_width*IQR)
df['Present_Price']=np.where(df['Present_Price']>upper_whisker,upper_whisker,np.where(df['Present_Price']<lower_whisker,lower_whisker,df['Present_Price']))

In [22]:
Q1 = df['Present_Price'].quantile(0.10)
Q3 = df['Present_Price'].quantile(0.90)
IQR = Q3 - Q1
whisker_width = 1.5
lower_whisker = Q1 - (whisker_width*IQR)
upper_whisker = Q3 + (whisker_width*IQR)
index=df['Present_Price'][(df['Present_Price']>upper_whisker)|(df['Present_Price']<lower_whisker)].index
df.drop(index,inplace=True)

In [23]:
outliers = find_outliers_IQR(df['Present_Price'])
print('number of outliers: '+ str(len(outliers)))
print('max outlier value: '+ str(outliers.max()))
print('min outlier value: '+ str(outliers.min()))

number of outliers: 0
max outlier value: nan
min outlier value: nan


In [24]:
# Data Encoding
le = LabelEncoder()
df['Fuel_Type'] = le.fit_transform(df['Fuel_Type'])
df['Seller_Type'] = le.fit_transform(df['Seller_Type'])
df['Transmission'] = le.fit_transform(df['Transmission'])
print(df)

    Car_Name  Year  Selling_Price  Present_Price  Kms_Driven  Fuel_Type  \
0       ritz  2014           3.35           5.59     27000.0          2   
1        sx4  2013           4.75           9.54     43000.0          1   
2       ciaz  2017           7.25           9.85      6900.0          2   
3    wagon r  2011           2.85           4.15      5200.0          2   
4      swift  2014           4.60           6.87     42450.0          1   
..       ...   ...            ...            ...         ...        ...   
296     city  2016           9.50          11.60     33988.0          1   
297     brio  2015           4.00           5.90     60000.0          2   
298     city  2009           3.35          11.00     87934.0          2   
299     city  2017          11.50          12.50      9000.0          1   
300     brio  2016           5.30           5.90      5464.0          2   

     Seller_Type  Transmission  Owner  
0              0             1      0  
1              0   

In [25]:
X = df.drop(['Car_Name','Selling_Price'],axis=1)
y = df['Selling_Price']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))


239
239
60
60


In [28]:
#Feature scaling
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [29]:
# Define Model: KNeighbors

# KNeighborsRegressor 
#K-Nearest Neighbours is considered to be one of the most intuitive machine learning algorithms since it is simple to understand and explain. 
#Additionally, it is quite convenient to demonstrate how everything goes visually. However, the kNN algorithm is still a common and 
#very useful algorithm to use for a large variety of classification and continuous problems.

knn = KNeighborsRegressor(n_neighbors=11, p=2,metric='euclidean')
knn.fit(X_train, y_train)

KNeighborsRegressor(metric='euclidean', n_neighbors=11)

In [30]:
y_pred = knn.predict(X_test)
y_pred

array([ 7.46      ,  7.73409091,  0.99545455,  6.45909091,  9.19863636,
        4.75909091,  4.90909091,  0.8       ,  7.73409091,  0.45909091,
        7.23863636,  0.52      ,  2.07181818,  0.80090909,  2.38181818,
        4.46818182,  0.77727273,  1.08636364, 12.69545455,  0.64181818,
        0.79090909,  1.29545455,  6.57272727,  2.46454545,  5.76363636,
        7.81      ,  7.53636364,  0.71636364,  3.71818182,  4.49545455,
        5.47272727,  2.48636364,  5.76363636,  3.35      ,  2.95454545,
        6.41818182,  0.75454545,  1.35272727,  0.66181818,  6.73136364,
        6.27727273,  7.61590909,  0.96818182,  2.96818182,  0.75454545,
        0.44272727,  8.38727273,  4.75909091,  6.61      ,  0.59      ,
        0.68818182,  0.56727273,  3.93636364,  7.97954545,  7.77727273,
        6.55      ,  3.48181818,  3.2       ,  8.79772727,  7.61363636])

In [31]:
# Evaluate Model
error_score = r2_score(y_test,y_pred)
print("R squared Error :", error_score)

R squared Error : 0.8212124834889327


In [42]:
# Define Model:Random Forest Regressor

# A random forest eradicates the limitations of a decision tree algorithm.
#It generates predictions without requiring many configurations in packages
#It’s more accurate than the decision tree algorithm.

RandomeForest = RandomForestRegressor(random_state=42)
RandomeForest.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [43]:
y_pred = RandomeForest.predict(X_test)
y_pred

array([ 9.3862 ,  8.1585 ,  0.4443 ,  6.8505 , 12.9545 ,  5.331  ,
        6.9905 ,  1.176  ,  7.9125 ,  0.2788 , 10.6275 ,  0.2093 ,
        0.2506 ,  0.4658 ,  4.4485 ,  5.3735 ,  0.6051 ,  1.1545 ,
       13.725  ,  0.5401 ,  0.5539 ,  2.5355 ,  6.2865 ,  2.229  ,
        6.237  ,  8.99345,  9.17325,  1.1485 ,  4.7266 ,  5.213  ,
        3.5545 ,  4.394  ,  6.2265 ,  2.6875 ,  3.268  ,  6.619  ,
        1.1595 ,  0.1661 ,  1.1615 , 12.752  ,  6.849  ,  9.2825 ,
        0.8982 ,  3.4786 ,  0.5466 ,  0.1978 ,  9.66255,  4.9965 ,
        3.9555 ,  0.2016 ,  0.597  ,  0.3325 , 12.7511 , 10.186  ,
        8.146  ,  7.178  ,  3.0765 ,  3.534  , 10.4765 ,  7.51725])

In [44]:
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
}

In [45]:
# Perform Grid Search with cross-validation
grid_search_rf = GridSearchCV(estimator= RandomeForest, param_grid=param_grid_rf, cv=5, scoring='neg_mean_squared_error')
grid_search_rf.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid={'max_depth': [None, 10, 20],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [46]:
best_rf_regressor = grid_search_rf.best_estimator_

In [47]:
# Evaluate Model
error_score = r2_score(y_test,y_pred)
print("R squared Error :", error_score)

R squared Error : 0.6428253009671525


In [48]:
# Define Model: Support Vector Machine 
#SVMs try to find the hyperplane in a high-dimensional space that maximally separates different classes or output values.

svm_regressor = SVR()

In [55]:
svm_regressor.fit(X_train, y_train)

SVR()

In [56]:
y_pred = svm_regressor.predict(X_test)
y_pred

array([ 8.21220454,  7.26723525,  0.87583252,  6.42496618,  5.96505917,
        4.5045227 ,  6.08603935,  0.99422336,  7.54342799,  0.72957898,
        7.54080648,  0.41862748,  1.09599073,  0.73593108,  3.26386638,
        5.08937979,  0.6349174 ,  1.14607381, 11.25802622,  0.49452187,
        0.50280581,  1.45112558,  5.78570476,  1.64564462,  5.44981942,
        8.59705215,  8.26504581,  0.69234832,  4.53307983,  4.69068433,
        3.86686138,  3.54168922,  5.45497015,  2.6576127 ,  2.99870819,
        6.34770202,  0.74650803,  3.02739306,  0.81000298,  5.03991893,
        6.05563545,  8.86631997,  0.97518661,  3.70214147,  0.53335598,
        0.38626538,  8.98625934,  4.3644292 ,  4.85804777,  0.56239869,
        0.513695  ,  0.42214981,  4.22175122,  9.26544178,  7.54052105,
        5.74027358,  3.43500858,  3.30673039,  9.37115479,  7.97184418])

In [57]:
param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'epsilon': [0.01, 0.1, 1],
}


In [58]:
grid_search_svm = GridSearchCV(estimator=svm_regressor, param_grid=param_grid_svm, cv=5, scoring='neg_mean_squared_error')
grid_search_svm.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10], 'epsilon': [0.01, 0.1, 1],
                         'kernel': ['linear', 'rbf', 'poly']},
             scoring='neg_mean_squared_error')

In [59]:
best_svm_regressor = grid_search_svm.best_estimator_

In [60]:
# Evaluate Model
error_score = r2_score(y_test,y_pred)
print("R squared Error :", error_score)

R squared Error : 0.898198712372273
